In [5]:
# import time
from technician_routing.routing_toolkit import RoutingConfig, VehicleRoutingEngine

In [13]:
config = RoutingConfig(
    'u:/data/darnley_tech_address.csv', 
    'u:/data/address_lat_long_121923.csv',
    40, # MPH
    55, # Minutes Per Service
    60*7 # Max Minutes per Route
)
engine = VehicleRoutingEngine(config).load_data()

Set 5269 addresses
Using uniform minutes on location of 55
Data Loaded


In [26]:
engine = engine \
    .set_driver_speed(45) \
    .set_addresses(engine.addresses[0:10]) \
    .set_time_on_location(65)

routes = engine.run(route_drivers=2) #[0,1,2, 3, 4, 5])
routes

Using driver speed of 45 MPH
Set 10 addresses
Using uniform minutes on location of 65
Calculated 12 node durations in 0.0
Running routing for 12 addresses and 2 drivers
Solved objective value 819


[[4, 6, 3, 0, 5, 1, 8, 4], [2, 9, 11, 10, 7, 2]]

In [27]:
for route in routes:
    
    travel_time = 0
    for start,end in zip(route, route[1:]):
        travel_time += engine.routing.distances[start, end]

    site_time = 0
    for location in route[1:-1]:
        site_time += engine.routing.demands[location]

    print(route, travel_time, site_time, travel_time + site_time)

[4, 6, 3, 0, 5, 1, 8, 4] 74.0 260 334.0
[2, 9, 11, 10, 7, 2] 95.0 260 355.0


In [ ]:
engine.routing.demands[20]

In [ ]:
engine.routing.distances[1,2]#.calculate_duration(1,2)

In [ ]:
for route in routes:
    

In [ ]:
def calculate_distances(self, mph=25):
        addresses = self.addresses['address'].unique()
        self.distances = np.zeros(shape=(len(addresses), len(addresses)))

        for i_idx, i in enumerate(addresses):
            i_coord = tuple(self.addresses[self.addresses['address'] == i][[
                'latitude', 'longitude'
            ]].values.tolist()[0])
            for j_idx, j in enumerate(addresses):
                j_coord = tuple(
                    self.addresses[self.addresses['address'] == j][[
                        'latitude', 'longitude'
                    ]].values.tolist()[0])
                self.distances[i_idx, j_idx] = hs.haversine(
                    i_coord, j_coord, unit=Unit.MILES) / (mph / 60)